In [3]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
import boto3
import io


Statistical tests to assess discriminative power of categorical variables in predicting Chronic Obstructive Pulmonary Disease class.
Based on Chi-squared hypothesis test and Cramer's V test.

In [9]:
s3 = boto3.client('s3')
bucket_name = "lneg-loka"
file_name = "patient_data_raw/patient_data_raw.csv"
obj = s3.get_object(Bucket=bucket_name, Key=file_name)
csv_string = obj['Body'].read().decode('utf-8')
df = pd.read_csv(io.StringIO(csv_string))

In [10]:
def cramers_v(confusion_matrix):
    """
    Compute Cramér's V for a contingency table.
    Uses the bias correction from Bergsma & Wicher (2013).
    """
    chi2, _, _, _ = chi2_contingency(confusion_matrix)
    n = confusion_matrix.sum().sum()
    if n == 0:
        return np.nan
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    # Bias correction
    phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    rcorr = r - ((r - 1) ** 2) / (n - 1)
    kcorr = k - ((k - 1) ** 2) / (n - 1)
    denom = min((kcorr - 1), (rcorr - 1))
    if denom <= 0:
        return 0.0
    return np.sqrt(phi2corr / denom)

In [11]:
def detect_categorical_columns(df, target_col, max_categories=6):
    """Auto-detect categorical columns:
       - object or category dtype
       - integer/float columns with small number of unique values (<= max_unique_for_numeric)
    """
    cats = []
    for col in df.columns:
        if col == target_col:
            continue
        if pd.api.types.is_object_dtype(df[col]) or pd.api.types.is_categorical_dtype(df[col]):
            cats.append(col)
            continue
        # numeric but small cardinality (likely categorical)
        if (pd.api.types.is_integer_dtype(df[col]) or pd.api.types.is_float_dtype(df[col])):
            nunique = df[col].nunique(dropna=True)
            if nunique <= max_categories:
                cats.append(col)
    return cats

In [12]:
def evaluate_features(df, categorical_features, target_col):
    rows = []
    for col in categorical_features:
        # Drop rows with missing values in either column
        subset = df[[col, target_col]].dropna()
        if subset.shape[0] == 0:
            rows.append({
                'feature': col,
                'chi2': np.nan,
                'p_value': np.nan,
                'dof': np.nan,
                'cramers_v': np.nan,
                'n_samples': 0,
                'note': 'no data after dropna'
            })
            continue

        contingency = pd.crosstab(subset[col], subset[target_col])
        # If contingency has any zero-row/column, chi2_contingency still works but may warn.
        try:
            chi2, p, dof, expected = chi2_contingency(contingency)
        except Exception as e:
            rows.append({
                'feature': col,
                'chi2': np.nan,
                'p_value': np.nan,
                'dof': np.nan,
                'cramers_v': np.nan,
                'n_samples': subset.shape[0],
                'note': f'chi2 error: {e}'
            })
            continue

        crv = cramers_v(contingency)
        # Check expected frequencies rule of thumb (many expected < 5 reduces reliability)
        low_expected = (expected < 5).sum()
        note = ''
        if low_expected > 0:
            note = f'{low_expected} expected cells < 5 (caution)'

        rows.append({
            'feature': col,
            'chi2': float(chi2),
            'p_value': float(p),
            'dof': int(dof),
            'cramers_v': float(crv),
            'n_samples': int(subset.shape[0]),
            'note': note
        })
    res_df = pd.DataFrame(rows).sort_values(by='cramers_v', ascending=False).reset_index(drop=True)
    return res_df

In [13]:
target_col = 'chronic_obstructive_pulmonary_disease'
df = df.drop('patient_id',axis=1)
cats = detect_categorical_columns(df,target_col,5)
res = evaluate_features(df,cats,target_col)
res

C:\Users\negra\AppData\Local\Temp\ipykernel_16308\276914831.py:10: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_object_dtype(df[col]) or pd.api.types.is_categorical_dtype(df[col]):


,feature,chi2,p_value,dof,cramers_v,n_samples,note
0,exercise_frequency,12.530375,0.051130,6,0.020219,7988,
1,diet_quality,13.493435,0.035836,6,0.019358,10000,
2,sex,1.412363,0.702639,3,0.000000,10000,
3,smoker,1.209976,0.750613,3,0.000000,10000,
4,diagnosis_code,11.634146,0.475491,12,0.000000,10000,
5,readmitted,1.260316,0.738577,3,0.000000,10000,
6,income_bracket,4.331325,0.631935,6,0.000000,10000,
7,education_level,5.337300,0.501335,6,0.000000,8960,
8,urban,0.463391,0.926857,3,0.000000,10000,
